# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096986


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:01<00:23,  1.18it/s]

Rendering models:  10%|█         | 3/30 [00:05<00:45,  1.70s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:35,  1.43s/it]

Rendering models:  20%|██        | 6/30 [00:07<00:24,  1.04s/it]

...

Rendering models:  70%|███████   | 21/30 [00:08<00:01,  6.90it/s]

Rendering models:  80%|████████  | 24/30 [00:08<00:00,  8.68it/s]

Rendering models:  87%|████████▋ | 26/30 [00:08<00:00, 10.23it/s]

Rendering models:  93%|█████████▎| 28/30 [00:08<00:00, 11.84it/s]

not-logged-in-3b0b929f4475fd276561    0.000799
OregonHiker                           0.000705
not-logged-in-07e6a4fb0102a346c69f    0.025756
Karl_Knopf                            0.001450
Kartoon1x                             0.000644
mwalsh868                             0.000558
not-logged-in-a4cf4d6534661ba8c651    0.000505
Quarkgc                               0.000900
not-logged-in-e7932b1588574fe4dae2    0.002478
24declanfm                            0.000633
not-logged-in-5704ea8f3a0f12b5bfb2    0.010044
not-logged-in-240a8f8da7f1bec60bbe    0.003585
Boogy33                               0.007823
41Letters                             0.000580
tominas                               0.001189
aidenr                                0.001055
aisha3540                             0.074410
rhondacard                            0.004536
helios1235                            0.000626
Lavadude                              0.036213
saferguson                            0.001157
Bambura      

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())